In [1]:

from __future__ import annotations
from datetime import datetime, timezone
from syft_event import SyftEvents
from syft_rds.api.api import API
from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext
import shutil


In [2]:
client = Client.load()

In [3]:
# input policy: constrain the inputs of a syftfunction
# output policy: How do we modify the outputs of a syftfunction, how many outputs can you create

In [4]:
from syft_core import SyftBoxURL
from syft_rds.models.code import CodeCreate, Code
from syft_rds.service.code_service import CodeService


box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  


@box.on_request("/apis/list")
def get_apis() -> list[str]:
    """Respond to a ping request."""
    return {SyftBoxURL.from_path(k, client.workspace): v for k, v in box._SyftEvents__rpc.items()}

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    
@box.on_request("/datasets/list")
def list_datasets() -> list[Dataset]:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.list_items()
    return res


@box.on_request("/code/create")
def create_code(code: CodeCreate) -> Code:
    """Respond to a ping request."""
    code_service = CodeService.from_context(context)
    res = code_service.create_item(code)
    return res


@box.on_request("/requests/create")
def create_request(request: RequestCreate) -> Request:
    """Respond to a ping request."""
    code_service = CodeService.from_context(context)
    res = code_service.create_item(code)
    return res

2025-02-11 11:13:52.121 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /apis/list


2025-02-11 11:13:52.123 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/create
2025-02-11 11:13:52.125 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/list
2025-02-11 11:13:52.127 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /code/create


In [5]:
app_data_dir = client.my_datasite / "apps" / context.box.app_name
shutil.rmtree(app_data_dir, ignore_errors=True)

In [6]:
conn = get_connection(box, mock=True)   

In [7]:
# DatasetCreate(name="my-dataset", description="my dataset").to_item()

In [8]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/apis/list",
    body={},
    expiry="5m",
    cache=True,
)

In [9]:
api = API.from_email(client.email, conn)

In [10]:
api.datasets.create(body=DatasetCreate(name="my-dataset", description="my dataset"))

Dataset(uid=UUID('450bed16-39b0-4f58-b65b-15a134745296'), name='my-dataset', description='my dataset')

In [16]:
datasets = api.datasets.list()


In [17]:
dataset = datasets[0]

Dataset(uid=UUID('450bed16-39b0-4f58-b65b-15a134745296'), name='my-dataset', description='my dataset')

In [ ]:
def my_code(dataset: Dataset) -> int:
    with open(dataset.path, "r") as f:
        data = pd.read_csv(f)
    return data 

In [ ]:
# open datasetes file
# get id from request, and get path from dataset
# read data from path

In [ ]:
api.requests.create(code=x, kwargs={"y": dataset})


In [12]:
api.code.create(body=CodeCreate(code_str="my-code"))

Code(uid=UUID('b574efc8-551c-4d90-acb5-e9446631062a'), code_str='my-code')

# Appendix

In [13]:
res

{'syft://koen@openmined.org/api_data/my-rds-app/rpc/apis/list': <function __main__.get_apis() -> 'list[str]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/create': <function __main__.create_dataset(dataset: 'DatasetCreate') -> 'Dataset'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/list': <function __main__.list_datasets() -> 'list[Dataset]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/code/create': <function __main__.create_code(code: 'CodeCreate') -> 'Code'>}

In [14]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

In [15]:
res

Dataset(uid=UUID('544774da-5dbe-4e02-aed3-4f17251aaf59'), name='my-dataset', description='my dataset')